# Running the JWST coronagraphic pipeline (coron3)
---
**Author**: Ben Sunnquist (bsunnquist@stsci.edu)| **Latest Update**: 10 April 2024

<a id='top'></a>
## Table of Contents
* [Introduction](#intro)
* [Pipeline Resources and Documentation](#resources)
* [Imports](#imports)
* [Data](#data)
* [Run the coron3 pipeline](#default_pipeline)
    * [Generate association file](#asn)
    * [Run the pipeline](#pipeline)
* [Run the coron3 pipeline using custom processing](#custom_pipeline)
    * [Correct bad pixels](#image_custom)
    * [Generate association file](#asn_custom)
    * [Run the pipeline with custom settings](#pipeline_custom)

<a id='intro'></a>
## Introduction

This notebook will show how to run the JWST stage 3 coronagraphic pipeline (coron3) on NIRCam data. The pipeline consists of the following 5 steps:
1. `outlier_detection` flags outlier pixels in all input images
2. `stack_refs` stacks all of the reference psf images together into a single product
3. `align_refs` aligns all of the reference psf images to the science target images
4. `klip` fits and subtracts a psf from each science target image
5. `resample` combines all of the psf-subtracted science images into a single product

<a id='resources'></a>
## Pipeline Resources and Documentation

* [JWST Coronagraphic Pipeline Documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_coron3.html) includes descriptions and parameters for all of the coronagraphic pipeline (coron3) steps.

* [JWST Pipeline Github](https://github.com/spacetelescope/jwst) contains the source code and installation instructions for the pipeline.

* [CRDS Reference Files](https://jwst-crds.stsci.edu/) is where all JWST reference files used by the pipeline are located.

* Submit a ticket to the [Help Desk](https://stsci.service-now.com/jwst?id=sc_cat_item&sys_id=27a8af2fdbf2220033b55dd5ce9619cd&sysparm_category=e15706fc0a0a0aa7007fc21e1ab70c2f) if you have any questions or problems regarding the pipeline.

<a id='imports'></a>
## Imports

In [ ]:
# How I made this environment:
# conda create -n jwst1125 python=3.11 notebook
# source activate jwst1125
# pip install jwst==1.12.5
# pip install astroquery

import os
os.environ['CRDS_PATH'] = './crds_cache/jwst_ops'  # set environment variables for reference files
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

from astropy.convolution import convolve, Gaussian2DKernel
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astroquery.mast import Mast
from astroquery.mast import Observations
import glob
import matplotlib.pyplot as plt
from scipy.ndimage import binary_dilation

import jwst
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3 import Asn_Lv3NRCCoron
from jwst.datamodels import dqflags
from jwst.pipeline import Coron3Pipeline
print('Using JWST Pipeline v{}'.format(jwst.__version__))

<a id='data'></a>
## Data

We'll use the NIRCam coronagraphic data from Early Release Science program 1386. The science target is HIP-65426 which is taken using 2 different roll angles in observations 2 and 3. These observations use the DEEP8 readout pattern with 2 integrations per exposure and 15 groups per intergration. The reference psf is HIP-68245 taken in observation 1 using a 9-POINT-CIRCLE dither pattern. This observation uses the MEDIUM8 readout pattern with 2 integrations per exposure and 4 groups per integration. All data is observed on the NRCALONG detector with the SUB320A335R subarray and uses the F444W filter and MASK335R round coronagraphic mask. We'll download the level 2b files, i.e. the _calints.fits files, as these are the necessary inputs into the coron3 pipeline.

In [ ]:
# Download the relevant data from MAST

params = {"columns" : "dataURI, filename",
          "filters" : [{"paramName" : "program", "values" : ['1386']},
                       {"paramName" : "observtn", "values" : ['1', '2', '3']},
                       {"paramName" : "exp_type", "values" : ['NRC_CORON']},
                       {"paramName" : "filter", "values" : ['F444W']},
                       {"paramName" : "productLevel", "values" : ['2b']}]}
t = Mast().service_request('Mast.Jwst.Filtered.Nircam', params)
for row in t:
    result = Observations().download_file(row['dataURI'], cache=False)

<a id='default_pipeline'></a>
## Run the coron3 pipeline

This section shows how to run the coron3 pipeline on NIRCam data using all of the default pipeline settings.

<a id='asn'></a>
### Generate association file

The input to the coron3 pipeline is an association file listing all of the relevant target and reference psfs, so we have to generate that first before running the pipeline.

In [ ]:
# Create an association file for the coron3 pipeline

cal_files = sorted(glob.glob('*calints.fits'))
asn = asn_from_list.asn_from_list(cal_files, rule=Asn_Lv3NRCCoron, product_name='HIP-65426_F444W')
for member in asn['products'][0]['members']:  # observations 2 and 3 are the science targets, observation 1 is the reference psf
    if fits.getheader(member['expname'])['IS_PSF']:
        member['exptype'] = 'psf'
    else:
        member['exptype'] = 'science'
asn.validity['has_psf']['validated'] = True  # set to True since psfs were added to the asn above
asn['asn_type'] = 'coron3'
with open('HIP-65426_F444W.json', 'w') as outfile:
    outfile.write(asn.dump()[1])
asn

<a id='pipeline'></a>
### Run the pipeline

Now, we can run the coron3 pipeline on the NIRCam data using the association file created above.

In [ ]:
# Run the coron3 pipeline

result = Coron3Pipeline.call('HIP-65426_F444W.json', save_results=True)

In [ ]:
# Inspect the psf-subtracted images generated in the coron3 pipeline run

files = sorted(glob.glob('*_nrcalong_a3001_psfsub.fits'))
grid = plt.GridSpec(2, 4, hspace=0.3, wspace=0.3)
fig = plt.figure(figsize=(14, 7))

# Plot the psf-subtracted science images for each roll and integration
for roll in range(len(files)):
    data = fits.getdata(files[roll])
    for integ in range(data.shape[0]):
        data_int = data[integ]
        ax = fig.add_subplot(grid[roll, integ])
        ax.imshow(data_int, vmin=-2, vmax=2, origin='lower', cmap='coolwarm')
        ax.set_title('Roll {} Int {}'.format(roll+1, integ+1))

# Plot the combined psf-subtracted science image
ax = fig.add_subplot(grid[0:2, 2:4])
data = fits.getdata('HIP-65426_F444W_i2d.fits')
ax.imshow(data, vmin=-2, vmax=2, origin='lower', cmap='coolwarm')
ax.set_title('Combined')

<a id='custom_pipeline'></a>
## Run the coron3 pipeline using custom processing

This section shows an example on how to incorporate custom processing and pipeline settings to improve the final coron3 pipeline products.

<a id='image_custom'></a>
### Correct bad pixels

The quality of the psf alignments, fittings, and subtractions can be negatively impacted by bad pixels. The coron3 pipeline uses the information in each images' data quality arrays to smooth these bad pixels' values to minimize their impact during processing; however, if any bad pixels are not flagged in the data quality arrays they will not be accounted for in this processing and can still negatively impact the coron3 pipeline results. In this section, we'll search for bad pixels in each image and flag any that aren't already flagged in the images' data quality arrays to ensure their impacts are minimized in the coron3 pipeline run. More details on the various data quality flags can be found [here](https://jwst-pipeline.readthedocs.io/en/latest/jwst/references_general/references_general.html#data-quality-flags).

In [ ]:
# Find new bad pixels in each image and flag them as OTHER_BAD_PIXEL in the data quality arrays. These bad pixels
# will be those with large differences between their original values and gaussian-smoothed values.

files = sorted(glob.glob('*_calints.fits'))
for file in files:
    h = fits.open(file)
    data = h['SCI'].data
    dq = h['DQ'].data
    for integ in range(data.shape[0]): 
        # Get the science and data quality arrays for this integration.
        data_int = data[integ]
        dq_int = dq[integ]
        
        # Gaussian-smooth the data. Known bad pixels not included in gaussian-fit; dq flags dilated to catch e.g. ipc.
        mean, med, stddev = sigma_clipped_stats(data_int, sigma=3, maxiters=5)
        data_int_bkgsub = data_int - med
        data_conv = convolve(data_int_bkgsub, Gaussian2DKernel(x_stddev=2), mask=binary_dilation(dq_int, iterations=1) != 0)

        # Flag pixels with large differences between original and gaussian-smoothed values. Also require their original
        # values to be 3 sigma from background to avoid flagging pixels with very small absolute values, and only flag 
        # those that are not already flagged as OTHER_BAD_PIXEL.
        diff = (data_int_bkgsub - data_conv) / data_conv
        diff_mean, diff_med, diff_stddev = sigma_clipped_stats(diff, sigma=3, maxiters=5)
        dq_int[(abs(diff) > 3*diff_stddev) & (abs(data_int_bkgsub) > 3*stddev) & (dq_int & dqflags.pixel['OTHER_BAD_PIXEL'] == 0)] += dqflags.pixel['OTHER_BAD_PIXEL']
        
        # Update the data quality array with the new bad pixels
        h['DQ'].data[integ] = dq_int

    # Write out the updated file
    h.writeto(file.replace('.fits', '_mod.fits'), overwrite=True)
    h.close()

<a id='asn_custom'></a>
### Generate association file

Now, we'll create an association file using these modified files for input into the coron3 pipeline.

In [ ]:
# Create an association file for the coron3 pipeline

cal_files = sorted(glob.glob('*calints_mod.fits'))
asn = asn_from_list.asn_from_list(cal_files, rule=Asn_Lv3NRCCoron, product_name='HIP-65426_F444W_mod')
for member in asn['products'][0]['members']:  # observations 2 and 3 are the science targets, observation 1 is the reference psf
    if fits.getheader(member['expname'])['IS_PSF']:
        member['exptype'] = 'psf'
    else:
        member['exptype'] = 'science'
asn.validity['has_psf']['validated'] = True  # set to True since psfs were added to the asn above
asn['asn_type'] = 'coron3'
with open('HIP-65426_F444W_mod.json', 'w') as outfile:
    outfile.write(asn.dump()[1])
asn

<a id='pipeline_custom'></a>
### Run the pipeline using custom settings

Finally, we'll run the coron3 pipeline. To ensure the pipeline treats the new bad pixels we flagged above as bad, we include the mnemonic we used to identify them (i.e. OTHER_BAD_PIXEL) in the bad_bits argument in the align_refs step because, by default, only pixels flagged DO_NOT_USE  are treated as bad. To further help catch any remaining bad pixels, we also reduce the SNR threshold in the outlier_detection step. All of the optional coron3 arguments and their descriptions can be found by clicking on the individual steps listed on the [JWST Coronagraphic Pipeline Documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_coron3.html) page and then clicking on the Arguments section.

In [ ]:
# Run the coron3 pipeline with custom pipeline settings

result = Coron3Pipeline.call('HIP-65426_F444W_mod.json', save_results=True, 
                             steps={'outlier_detection': {'snr': '4.5 3.5'},
                                    'align_refs': {'bad_bits': 'DO_NOT_USE, OTHER_BAD_PIXEL'}})

In [ ]:
# Inspect the psf-subtracted images generated in the custom coron3 pipeline run

files = sorted(glob.glob('*_nrcalong_calints_mod_a3001_psfsub.fits'))
grid = plt.GridSpec(2, 4, hspace=0.3, wspace=0.3)
fig = plt.figure(figsize=(14, 7))

# Plot the psf-subtracted science images for each roll and integration
for roll in range(len(files)):
    data = fits.getdata(files[roll])
    for integ in range(data.shape[0]):
        data_int = data[integ]
        ax = fig.add_subplot(grid[roll, integ])
        ax.imshow(data_int, vmin=-2, vmax=2, origin='lower', cmap='coolwarm')
        ax.set_title('Roll {} Int {}'.format(roll+1, integ+1))

# Plot the combined psf-subtracted science image
ax = fig.add_subplot(grid[0:2, 2:4])
data = fits.getdata('HIP-65426_F444W_mod_i2d.fits')
ax.imshow(data, vmin=-2, vmax=2, origin='lower', cmap='coolwarm')
ax.set_title('Combined')

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 